<div align="center", style="color: purple">

# Zadanie 1

</div>

In [223]:
import numpy as np

In [224]:
# Tworzy losowego osobnika
def random_individual(n: int, x_min: float = -5.0, x_max: float = 5.0):
    return {
        'x': np.random.uniform(x_min, x_max, size=n),
        'o': np.random.uniform(0.1, 1.0, size=n)   # sigma
    }

# Tworzy populację
def random_population(mu: int, n: int, x_min: float = -5.0, x_max: float = 5.0):
    return np.array([random_individual(n, x_min, x_max) for _ in range(mu)], dtype=object)

# Ocena populacji
def population_evaluation(P, F):
    return np.array([F(ind['x']) for ind in P])

# Selekcja rodziców (z powtórzeniami!)
def parent_selection(P, lam: int):
    idx = np.random.choice(len(P), lam, replace=True)
    return P[idx]

# Mutacja gaussowska
def mutation(Pc, tau, tau0):
    lam = len(Pc)
    n = len(Pc[0]['x'])

    for i in range(lam):
        indiv = Pc[i]
        x = indiv['x']
        o = indiv['o']

        # Mutacja parametrów strategii
        N0 = np.random.normal()
        Ni = np.random.normal(size=n)

        o_new = o * np.exp(tau0 * N0 + tau * Ni)

        # Mutacja wartości x
        x_new = x + o_new * np.random.normal(size=n)

        indiv['x'] = x_new
        indiv['o'] = o_new

    return Pc

import copy

def replacement_plus(P, Pc, mu, F):
    combined = np.array([copy.deepcopy(ind) for ind in list(P) + list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

# Replacement: ES(mu, lambda)
def replacement_comma(Pc, mu, F):
    combined = np.array([copy.deepcopy(ind) for ind in list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

In [225]:
ITER = 10000

# Strategia ewolucyjna ES(mu + lambda)
def ES_plus(mu, lam, tau, tau0, F, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0)
        P = replacement_plus(P, Pc, mu, F)

    fitness = population_evaluation(P, F)
    return P[np.argmax(fitness)]

# Strategia ewolucyjna ES(mu, lambda)
def ES_comma(mu, lam, tau, tau0, F, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0)
        P = replacement_comma(Pc, mu, F)

    fitness = population_evaluation(P, F)
    return P[np.argmax(fitness)]

In [226]:
def sphere(x):
    return -np.sum(x**2)  # minimalizacja -> dajemy minus

In [ ]:
D = 100
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=sphere, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-sphere(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=sphere, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-sphere(best)}')

Najlepszy osobnik: [ 9.88763825e-03 -1.00920557e-02 -6.66582924e-03 -1.18344836e-02
 -2.29810379e-02  7.12908662e-03  1.09508341e-03 -1.04308692e-02
  3.45287606e-02 -1.37354652e-05  1.42900646e-01  1.29644657e-02
 -1.44297035e-02 -1.24250039e-02 -2.93258121e-03  2.69560305e-02
 -4.86619609e-02 -2.75843657e-03  3.11108521e-02  2.53802788e-03
  6.93935213e-03  4.69764447e-03  4.75611120e-02  4.07361689e-03
 -1.01321232e-02  5.68454322e-03 -1.59518513e+00  1.38299300e-02
  6.40511789e-03  7.12528883e-02  5.25198689e-03 -1.16956207e-02
 -9.78265028e-04  6.97660837e-03 -2.47886837e-02  1.97086476e-02
 -1.35593536e-03  5.30543841e-03  2.10510552e-02 -1.11339335e-02
  3.12806477e-03  9.21941625e-03 -5.98041213e-03  1.48392729e-02
 -2.69574616e-02  7.59699082e-03  4.51669894e-01 -1.14035341e-03
  9.81212194e-03  2.46694186e-02 -1.37612716e-03  1.56734432e-02
 -1.77078407e-03  1.30015757e-02  1.64697276e-02 -1.23698667e-01
  2.96922127e-02  2.70883238e-02 -4.14422904e-03  1.65968939e-02
  8.75

In [228]:
def easom(x):
    x1, x2 = x[0], x[1]
    return np.cos(x1) * np.cos(x2) * np.exp(-((x1 - np.pi)**2 + (x2 - np.pi)**2))

D = 2
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=easom, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-easom(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=easom, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-easom(best)}')

Najlepszy osobnik: [3.14159266 3.14159266]
Wartość, którą osiąga: -1.0

Najlepszy osobnik: [3.14159265 3.14159266]
Wartość, którą osiąga: -1.0


In [229]:
def dixon_price(x):
    x = np.array(x)
    term1 = (x[0] - 1)**2
    term2 = np.sum(np.arange(2, len(x)+1) * (2*x[1:]**2 - x[:-1])**2)
    return -(term1 + term2)

D = 100
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=dixon_price, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-dixon_price(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=dixon_price, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-dixon_price(best)}')

Najlepszy osobnik: [ 6.03116480e-01  4.62953657e-01  4.71194999e-01  5.63720339e-01
  6.41748127e-01  7.40247116e-01 -3.99280495e-01 -3.99982378e-01
  2.33628437e-03  6.43257071e-02  1.86539888e-01 -4.39499462e-03
  9.34623547e-03  6.88678253e-02  1.78831303e-01 -3.04354366e-01
  4.82831625e-02  1.94036234e-01  3.27687645e-01  4.07389953e-01
 -3.09182542e-01  6.66323625e-03 -1.41303188e-04  8.07102315e-03
  4.38160802e-02  1.94634030e-01  3.34645924e-01  4.05182628e-01
  4.37488077e-01  3.74754072e-01  2.59479147e-01  9.94891945e-02
  1.18123308e-01  1.93452575e-01  2.19496105e-01 -8.42480070e-03
  4.59114815e-03  6.25321490e-03 -2.79852927e-02  1.11387975e-01
  2.64095775e-01  3.79082893e-01  4.55629109e-01  5.05783684e-01
  5.45494600e-01  4.48065728e-01 -3.11701534e-01 -9.13594444e-04
  5.03052965e-03  6.65237087e-03  1.52695255e-02  9.18436979e-02
  1.58925644e-01  8.59040677e-04  1.78839241e-02  1.20082322e-01
  2.48119091e-01  3.44372824e-01  4.06313704e-01  4.33325174e-01
  4.30

In [230]:
def rastrigin(x):
    x = np.array(x)
    n = len(x)
    return -(10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x)))

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=rastrigin, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-rastrigin(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=rastrigin, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-rastrigin(best)}')

Najlepszy osobnik: [ 3.97927534e+00  1.65409723e-04  9.95261545e-01 -1.98958439e+00
  9.91427205e-01 -8.98551285e-05 -9.97959753e-01  2.24976700e-04
  9.94918475e-01 -9.94962525e-01 -9.94901661e-01 -9.94774013e-01
 -2.88089543e-04  2.49874592e-05  9.95070681e-01  1.99012234e+00
 -4.00963596e-05 -9.64521475e-06  9.94886318e-01  4.93549801e-04]
Wartość, którą osiąga: 32.837988869193055

Najlepszy osobnik: [-2.98488791e+00  4.32857909e-05 -1.99045345e+00 -2.98477244e+00
  6.54379623e-05  4.97359927e+00  9.94822541e-01 -9.95051930e-01
  9.94736514e-01 -9.83651071e-05  9.94935948e-01  1.99042193e+00
 -9.95088102e-01  1.98996411e+00  1.08327555e+00 -3.97982821e+00
  1.99000178e+00  5.97002614e+00  2.98491883e+00  1.98989582e+00]
Wartość, którą osiąga: 130.86054031532723


In [231]:
def schwefel(x):
    x = np.array(x)
    n = len(x)
    return -(418.9829 * n - np.sum(x * np.sin(np.sqrt(np.abs(x)))))

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=schwefel, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-schwefel(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=schwefel, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-schwefel(best)}')

Najlepszy osobnik: [5.23921495 5.23930375 5.23919686 5.23919958 5.23919985 5.23919809
 5.23854474 5.23919976 5.23920029 5.23919962 5.23924506 5.23919949
 5.23919773 5.23919568 5.2391963  5.23914179 5.2392008  5.23919815
 5.23919587 5.23919952]
Wartość, którą osiąga: 8300.751967584181

Najlepszy osobnik: [5.23914518 5.2391932  5.24167017 5.23922074 5.23935123 5.23908082
 5.23936097 5.23919844 5.2389101  5.23919029 5.2392112  5.2392119
 5.23908185 5.23919349 5.23954005 5.23868491 5.23907893 5.23920388
 5.23919736 5.238232  ]
Wartość, którą osiąga: 8300.751969029085


In [232]:
def griewank(x):
    x = np.array(x)
    n = len(x)

    sum_term = np.sum(x**2) / 4000
    prod_term = np.prod(np.cos(x / np.sqrt(np.arange(1, n+1))))

    return -(1 + sum_term - prod_term)

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=griewank, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-griewank(best)}\n')

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=griewank, n=D)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-griewank(best)}')

Najlepszy osobnik: [ 8.88301019e-07  1.53752578e-05  8.82220667e-07  3.07029407e-06
 -2.08380514e-03 -2.32287077e-05  9.35168767e-06 -2.86037457e-06
  2.37996822e-06  2.40023771e-06 -1.47344604e-06  8.73376093e-06
  3.82135670e-07  1.83767617e-02 -9.51679704e-06  1.77528738e-03
  1.03772135e-06  1.53676537e-05 -7.60294025e-06  4.28553115e-06]
Wartość, którą osiąga: 1.2680016812538497e-05

Najlepszy osobnik: [ 1.21091889e-07  4.43845908e+00 -5.12786841e-06 -1.26146269e-07
  1.00783896e-07 -2.93976305e-07 -2.15872104e-07  8.85036313e+00
 -5.12478192e-07 -3.72261625e-07  2.51490072e-06  3.21642817e-07
  2.80731552e-06  1.46711524e-05 -1.54841110e-07  1.30525099e-03
  3.90194214e-07 -3.21219087e-07 -1.14755022e-08  1.73544037e-04]
Wartość, którą osiąga: 0.02459049194231344


# Problemy z ograniczeniami

In [ ]:
import numpy as np

# --------------------------
# Funkcja celu G2 (maksymalizacja)
# --------------------------
def g2_objective(x):
    x = np.array(x)
    n = len(x)

    numerator = np.sum(np.cos(x)**4) - 2 * np.prod(np.cos(x)**2)
    denominator = np.sqrt(np.sum((np.arange(1, n+1) * x**2)))

    return numerator / denominator


# --------------------------
# Ograniczenia (<= 0)
# --------------------------

def g2_constraint_1(x):
    # g1(x) = Π xi  + 0.75 <= 0
    return np.prod(x) + 0.75


def g2_constraint_2(x):
    # g2(x) = Σ xi - 7.5n <= 0
    x = np.array(x)
    n = len(x)
    return np.sum(x) - 7.5 * n
